# Get data from PKIDB

PKIDB is a curated, annotated and regularly updated database of protein kinase inhibitors in clinical trials. 

This notebooks fetches the PKI table from the PKIDB website to be used in our kinase projects.

In [1]:
from datetime import datetime

import pandas as pd

# In order to be able to call this PKIDB function from other places, we load it from a python file
from pkidb_download import get_data_from_pkidb

In [2]:
less pkidb_download.py

import requests
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd

def get_data_from_pkidb():
    """
    Get PKI table from PKIDB website.
    """
    
    # Request content from URL
    r = requests.get("http://www.icoa.fr/pkidb/index.html")
    r.raise_for_status()

    # Transform into html
    html = BeautifulSoup(r.text, features='html.parser')

    # Get table
    table = html.find('table', id="customers")

    # Get table column names
    header = table.find('thead')
    header = [i.text for i in header.find_all('th')]

    # Get table rows
    body = table.find('tbody')
    rows = body.find_all('tr')    
    # Separator needed to replace <br> with space, otherwise 'a</br>b' becomes 'ab' instead of 'a b'
    rows = [[cell.get_text(separator=u' ') for cell in row.find_all('td')] for row in rows]  
    
    # Cast data to DataFrame
    pkidb_ligands = pd.DataFrame(rows, columns=header)
    pkidb_ligands.rename(columns={i: i.replace(' ', '_') for i in pkidb_liga

## Get PKIDB data

In [3]:
pkidb_ligands = get_data_from_pkidb()

In [4]:
pkidb_ligands.to_csv(f'results/pkidb_{datetime.now().strftime("%Y%m%d")}.csv', index_label=None)